In [ ]:
import pandas as pd

In [ ]:
url1='https://raw.githubusercontent.com/jdodway/F1Anlysis/main/F1QueryResults/F1QualyAllFastest.csv'
url2='https://raw.githubusercontent.com/jdodway/F1Anlysis/main/F1QueryResults/F1RaceFastestLap.csv'
url3='https://raw.githubusercontent.com/jdodway/F1Anlysis/main/F1QueryResults/F1RaceTimeline2012.csv'
url4='https://raw.githubusercontent.com/jdodway/F1Anlysis/main/F1QueryResults/F1Qual2012.csv'
url5='https://raw.githubusercontent.com/jdodway/F1Anlysis/main/F1QueryResults/f1driversnation.csv'

In [ ]:
AllTimeFastQ = pd.read_csv(url1)
AllTimeFastR = pd.read_csv(url2)
Races2012 = pd.read_csv(url3)
Qual2012 = pd.read_csv(url4)
DriversNations = pd.read_csv(url5)

In [ ]:
import numpy as np

In [ ]:
AllTimeFastQ[['q3','q2']] = AllTimeFastQ[['q3','q2']].replace(r'\N',np.nan) 
AllTimeFastQ

In [ ]:
Qual2012[['q3']] = Qual2012[['q3']].replace(r'\N',np.nan) 
Qual2012

In [ ]:
AllTimeFastR.head()

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import lxml

In [ ]:
laptimes = pd.DataFrame(columns = ['Year','Track', 'FastestLap'])
def scrapefastlap():
    page = 1950
    global laptimes
    targettracks = ['Great Britain','Belgium', 'Italy','Monaco']
    while page != 2004:
        url = f'https://www.formula1.com/en/results.html/{page}/fastest-laps.html'
        response = requests.get(url)
        soup = bs(response.text,'lxml')
        table = soup.find('table',class_='resultsarchive-table')
        for row in table.tbody.find_all('tr'):
            rows = row.find_all('td')
            if any(x in row.find_all('td')[1].text for x in targettracks):
                track = rows[1].text
                fastlap = rows[4].text
                
                laptimes.loc[len(laptimes.index)] = [page,track, fastlap]
        page = page + 1

In [ ]:
scrapefastlap()

In [ ]:
laptimes

In [ ]:
AllTimeFastR.rename({'circuitRef':'Track','year':'Year'},axis='columns',inplace=True)
AllTimeFastR

In [ ]:
RaceFastLaps = pd.concat([laptimes, AllTimeFastR.drop('circuitName',axis=1)])
RaceFastLaps

In [ ]:
RaceFastLaps[['Track']] = RaceFastLaps[['Track']].replace(['spa', 'monza', 'silverstone', 'monaco'], ['Belgium', 'Italy', 'Great Britain','Monaco'])
RaceFastLaps

In [ ]:
RaceFastLaps['FastestLap'] = pd.to_timedelta(RaceFastLaps['FastestLap'].apply(lambda x: f'00:{x}')).dt.total_seconds().map('{:,.2f}'.format)
RaceFastLaps['FastestLap'] = RaceFastLaps['FastestLap'].astype(float)
RaceFastLaps

In [ ]:
pivRaceLaps = RaceFastLaps.pivot(index='Year', columns = 'Track', values = 'FastestLap')
pivRaceLaps
pivRaceLaps.to_csv('pivRaceLaps.csv')